In [1]:
import cv2
import numpy as np
import json

In [2]:
img_name = r"BoothVoterList_A4_Ward_9_Booth_1_img001.jpg"
grid_path = (f"../data/output/{img_name}")

In [3]:
def cluster_positions(positions, gap=5):
    clusters = []
    current = [positions[0]]
    print(current)
    for p in positions[1:]:
        if p - current[-1] <= gap:
            current.append(p)
        else:
            clusters.append(int(np.mean(current)))
            current = [p]

    clusters.append(int(np.mean(current)))
    return clusters

In [4]:
grid = cv2.imread(grid_path, cv2.IMREAD_GRAYSCALE)
if grid is None:
    raise ValueError("Could not load image")

_, grid = cv2.threshold(grid, 127, 255, cv2.THRESH_BINARY)
h, w = grid.shape

In [5]:
# ---- Horizontal lines ----
h_proj = np.sum(grid, axis=1)
h_thresh = 0.5 * np.max(h_proj)
ys = np.where(h_proj > h_thresh)[0]
row_lines = cluster_positions(ys)

# ---- Vertical lines ----
v_proj = np.sum(grid, axis=0)
v_thresh = 0.5 * np.max(v_proj)
xs = np.where(v_proj > v_thresh)[0]
col_lines = cluster_positions(xs)

[np.int64(335)]
[np.int64(59)]


In [6]:
rectangles = {}
rect_id = 1

for r in range(len(row_lines) - 1):
    for c in range(len(col_lines) - 1):
        x1, x2 = col_lines[c], col_lines[c + 1]
        y1, y2 = row_lines[r], row_lines[r + 1]

        rectangles[f"rectangle_{rect_id}"] = {
            "top_left": [x1, y1],
            "top_right": [x2, y1],
            "bottom_right": [x2, y2],
            "bottom_left": [x1, y2]
        }
        rect_id += 1

In [7]:
output = {
    "image_name": grid_path,
    "total_rectangles": len(rectangles),
    "coordinates": rectangles
}

json_path = grid_path + ".json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(output, f, indent=4)

print(f"[OK] Saved → {json_path}")

rectangles = output['coordinates']
print(rectangles)

[OK] Saved → ../data/output/BoothVoterList_A4_Ward_9_Booth_1_img001.jpg.json
{'rectangle_1': {'top_left': [60, 336], 'top_right': [837, 336], 'bottom_right': [837, 630], 'bottom_left': [60, 630]}, 'rectangle_2': {'top_left': [837, 336], 'top_right': [1615, 336], 'bottom_right': [1615, 630], 'bottom_left': [837, 630]}, 'rectangle_3': {'top_left': [1615, 336], 'top_right': [2391, 336], 'bottom_right': [2391, 630], 'bottom_left': [1615, 630]}, 'rectangle_4': {'top_left': [60, 630], 'top_right': [837, 630], 'bottom_right': [837, 925], 'bottom_left': [60, 925]}, 'rectangle_5': {'top_left': [837, 630], 'top_right': [1615, 630], 'bottom_right': [1615, 925], 'bottom_left': [837, 925]}, 'rectangle_6': {'top_left': [1615, 630], 'top_right': [2391, 630], 'bottom_right': [2391, 925], 'bottom_left': [1615, 925]}, 'rectangle_7': {'top_left': [60, 925], 'top_right': [837, 925], 'bottom_right': [837, 1220], 'bottom_left': [60, 1220]}, 'rectangle_8': {'top_left': [837, 925], 'top_right': [1615, 925], '

In [15]:
image_path = f"../data/{img_name}"   # img here is filename only
image = cv2.imread(image_path)

if image is None:
    raise ValueError(f"Failed to load image: {image_path}")

for rect_name, rect in rectangles.items():
    # Extract coordinates
    x1, y1 = rect["top_left"]
    x2, y2 = rect["bottom_right"]

    # Ensure integers
    x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])

    # Crop
    crop = image[y1:y2, x1:x2]

    # -------------------------
    # SHOW cropped IMAGE
    # -------------------------
    cv2.imshow("Grid Points", crop)
    cv2.waitKey(100)
    cv2.destroyAllWindows()